In [1]:
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from meta import OUT
from torch_geometric_temporal.nn.attention import STConv,GMAN,ASTGCN
from torch_geometric_temporal.nn.recurrent import DCRNN
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from utils import od_dataloader,ODLoss,set_seed,masked_mse
from config import get_configs
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from lib import train, test

args = get_configs()
args.model_name = 'astgcn'

In [2]:
seeds = [42, 3407, 114514]

se = torch.Tensor(np.load(args.se_path)).to(args.device)
pe = torch.Tensor(pd.read_csv(args.pe_path,index_col=0).values).to(args.device)
adj = torch.cuda.LongTensor(pd.read_csv('../data/processed data/adj.csv',index_col=0).values)
ed = coo_matrix(adj.cpu())
edge_index = torch.cuda.LongTensor(np.array([ed.row, ed.col]))
train_dataloader, val_dataloader, test_dataloader = od_dataloader(args)

train number:6965
validation number:871
test number:871


In [3]:
class STGCN(nn.Module):
    def __init__(self, n_feature, num_nodes, in_seq_len, d_model):
        super(STGCN, self).__init__()
        self.n_feature = n_feature
        self.num_nodes = num_nodes
        self.in_seq_len = in_seq_len
        self.d_model = d_model
        self.stgcn = STConv(self.num_nodes, self.n_feature, self.d_model,self.d_model, 3, 1)
        self.fc = nn.Linear(20,self.in_seq_len)
        self.out = OUT(self.num_nodes, self.in_seq_len, self.in_seq_len, self.d_model)

    def forward(self, x, te, se, pe, adj, edge_index):      
        B,T,N,D = x.size()
        h = self.stgcn(x,edge_index)
        h = h.permute(0,2,3,1)
        h = self.fc(h)
        h = h.permute(0,3,1,2)
        h = self.out(h)
        return h

In [4]:
class ODASTGCN(nn.Module):
    def __init__(self, n_feature, num_nodes, in_seq_len, d_model):
        super(ODASTGCN, self).__init__()
        self.n_feature = n_feature
        self.num_nodes = num_nodes
        self.in_seq_len = in_seq_len
        self.d_model = d_model
        self.astgcn = ASTGCN(nb_block=2, in_channels=132, K=3, nb_chev_filter=64, nb_time_filter=64, time_strides=2, num_for_predict=24, len_input=24, num_of_vertices=66)
        self.fc = nn.Linear(1,self.d_model)
        self.out = OUT(self.num_nodes, self.in_seq_len, self.in_seq_len, self.d_model)

    def forward(self, x, te, se, pe, adj, edge_index):
        x = x.permute(0,2,3,1) # (B,N,D,T)
        h = self.astgcn(x,edge_index)
        h = h.unsqueeze(-1)
        h = h.permute(0,2,1,3) # (B,T,N,1)
        h = self.fc(h)
        h = self.out(h)
        return h
    

In [ ]:
for seed in seeds:
    args.seed = seed
    set_seed(args.seed)
    print("*" * 10)
    print(f"start seed {args.seed}")
    print("*" * 10)
    model = ODASTGCN(args.n_feature,args.num_nodes,args.in_seq_len,args.d_model)
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.0005)
    criterion = ODLoss
    scheduler = StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
    train(model,train_dataloader,se,pe,edge_index,val_dataloader,optimizer,criterion,scheduler,args)
    torch.cuda.empty_cache()
    # mse,rmse,mae,mape,_,_ = test(model,test_dataloader,se,pe,criterion,args,True)
    # print(f"test mse is {mse}")
    # print(f"test rmse is {rmse}")
    # print(f"test mae is {mae}")
    # print(f"test mape is {mape}")

**********
start seed 42
**********
Epoch: 00, Loss: 3.9421
test loss: 4.0531
Epoch: 01, Loss: 3.6802
Epoch: 02, Loss: 3.6002
Epoch: 03, Loss: 3.5441
test loss: 3.9483
